In [1]:
import numpy as np
import tensorflow as tf
learn = tf.contrib.learn
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
mnist = learn.datasets.load_dataset('mnist')
data = mnist.train.images
labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [3]:
feature_columns = learn.infer_real_valued_columns_from_input(data)
classifier = learn.LinearClassifier(feature_columns=feature_columns, n_classes=10)
classifier.fit(data, labels, batch_size=100, steps=1000)
classifier.evaluate(eval_data, eval_labels)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d1f24cac8>, 'save_checkpoints_secs': 600, 'save_summary_steps': 100, '_task_id': 0, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_type': None, 'keep_checkpoint_max': 5, 'save_checkpoints_steps': None, '_is_chief': True, 'tf_random_seed': None, '_num_ps_replicas': 0, '_master': '', '_evaluation_master': '', '_environment': 'local'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and bat

{'accuracy': 0.92159998, 'global_step': 1000, 'loss': 0.28329819}

In [4]:
def conv_model(x, is_training):
    # Reshape X to 4-D tensor: [batch, width, height, channels]
    x = tf.reshape(x, [-1, 28, 28, 1])
    # first conv layer will compute 32 features for each 5x5 patch
    h_conv1 = tf.contrib.layers.conv2d(x, num_outputs=32, kernel_size=[5, 5])
    h_pool1 = tf.contrib.layers.max_pool2d(h_conv1, [2, 2], padding='SAME')
    h_conv2 = tf.contrib.layers.conv2d(h_pool1, num_outputs=64,
                                       kernel_size=[5, 5])
    h_pool2 = tf.contrib.layers.max_pool2d(h_conv2, [2, 2], padding='SAME')
    # Reshape tensor into a batch of vectors
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    # Densely connected layer with 1024 neurons
    h_fcl = tf.contrib.layers.relu(h_pool2_flat, 1024)
    h_dropout = tf.contrib.layers.dropout(
        h_fcl, keep_prob=0.5, is_training=is_training)
    # Logits
    logits = tf.contrib.layers.linear(h_dropout, 10)
    return logits  

In [11]:
def model_fn(x, y, mode):
    logits = conv_model(x, mode == tf.contrib.learn.ModeKeys.TRAIN)
    sw = tf.train.SummaryWriter('./logs2', graph=tf.get_default_graph())
    if mode != tf.contrib.learn.ModeKeys.INFER:
        loss = tf.contrib.losses.softmax_cross_entropy(logits, y)
        #sw = tf.train.SummaryWriter('./logs2', graph=tf.get_default_graph())
        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            train_op = tf.contrib.layers.optimize_loss(loss,
                                                       tf.contrib.framework.get_global_step(),
                                                       0.001, 'SGD')
    
    return logits, loss, train_op

In [12]:
classifier = learn.Classifier(model_fn=model_fn, n_classes=10)
classifier.fit(data, labels, batch_size=100, steps=20000)
classifier.evaluate(eval_data, eval_labels)

Instructions for updating:
Please use Estimator directly.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d197699e8>, 'save_checkpoints_secs': 600, 'save_summary_steps': 100, '_task_id': 0, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_type': None, 'keep_checkpoint_max': 5, 'save_checkpoints_steps': None, '_is_chief': True, 'tf_random_seed': None, '_num_ps_replicas': 0, '_master': '', '_evaluation_master': '', '_environment': 'local'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by m

ValueError: Shapes (?, 10) and (?,) are incompatible